In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"


In [2]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')
# データフレームを表示して確認
df.head()

,カテゴリー,商品コード,商品名,売上日,単価,数量,原価
0,食品,1001,りんご,2023-01-01,200,50,120
1,食品,1002,バナナ,2023-01-01,150,100,80
2,食品,1003,牛乳,2023-01-02,180,80,100
3,衣服,2001,Tシャツ,2023-01-02,1500,20,800
4,衣服,2002,ジーンズ,2023-01-03,5000,10,2500


In [3]:
# 2. データをLLM用にテキスト形式に変換
# データフレーム全体を文字列に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"
# 表示して確認
print(prompt_text)

売上データ:
    カテゴリー 商品コード      商品名         売上日    単価   数量    原価
0      食品  1001      りんご  2023-01-01   200   50   120
1      食品  1002      バナナ  2023-01-01   150  100    80
2      食品  1003       牛乳  2023-01-02   180   80   100
3      衣服  2001     Tシャツ  2023-01-02  1500   20   800
4      衣服  2002     ジーンズ  2023-01-03  5000   10  2500
..    ...   ...      ...         ...   ...  ...   ...
235    衣服  2077   レインパンツ  2023-04-28  2000   18  1000
236    食品  1085      ザクロ  2023-04-29   600   40   300
237   日用品  3077    バスブラシ  2023-04-29   400   60   200
238    衣服  2078  レインシューズ  2023-04-30  2500   15  1250
239    食品  1086    ココナッツ  2023-04-30   300   80   150

[240 rows x 7 columns]
この売上データの傾向を分析してください。


In [4]:
# 3. OpenAI APIの呼び出し

# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())

この売上データを分析すると、いくつかの興味深い傾向やインサイトを得ることができます。以下にいくつかの観点から分析してみます。

### 1. カテゴリー別の売上分析
- **食品、衣服、日用品**の3つのカテゴリーが含まれています。
- 特に食品カテゴリーにおける売上が高いことが予想されますが、実際に何が売れているかを確認するためには、カテゴリごとの総売上や数量を集計する必要があります。
  
### 2. 時系列分析
- 売上日は2023年1月から4月までのデータがあります。この期間での売上の動向を月ごとや週ごとに集計することで、季節性やトレンドを分析できます。
- 例えば、特定の月に売上が急増した場合、その時期に行ったプロモーションやイベントが原因かもしれません。

### 3. 商品パフォーマンス
- 商品ごとの売上（単価 × 数量）を計算して、各商品のパフォーマンスを見極めることができます。
- 売上の高い商品や利益率の高い商品を特定することで、在庫管理やマーケティング戦略に活用できます。

### 4. 利益分析
- 売上から原価を引くことで、各商品の利益を計算できます。これにより、利益率の高い商品や、原価が高いが売上が低い商品を特定できます。

### 5. 顧客セグメンテーション
- 売上データが顧客情報と結びつけられる場合、顧客の購入傾向を分析し、ターゲットセグメントを特定できます。例えば、特定のカテゴリーの商品を好む顧客セグメントがあるかもしれません。

### 具体的なアクションプラン
1. **データの可視化**: 売上や利益のトレンドをグラフで可視化し、重要なパターンを把握します。
2. **不振商品の特定**: 売上が低迷している商品の改善策を検討します。
3. **プロモーション戦略**: 売上が急増した時期の施策を分析し、今後のプロモーション戦略に活かします。

以上の観点でデータをさらに深く分析することで、企業の成長に寄与するインサイトを得ることができるでしょう。具体的な数値やデータを基にした詳細な分析が必要です。


In [5]:
# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])
print(df_out)

                                                   結果
0   この売上データを分析すると、いくつかの興味深い傾向やインサイトを得ることができます。以下にい...
1                                                    
2                                  ### 1. カテゴリー別の売上分析
3                   - **食品、衣服、日用品**の3つのカテゴリーが含まれています。
4   - 特に食品カテゴリーにおける売上が高いことが予想されますが、実際に何が売れているかを確認す...
5                                                    
6                                        ### 2. 時系列分析
7   - 売上日は2023年1月から4月までのデータがあります。この期間での売上の動向を月ごとや週...
8   - 例えば、特定の月に売上が急増した場合、その時期に行ったプロモーションやイベントが原因かも...
9                                                    
10                                   ### 3. 商品パフォーマンス
11   - 商品ごとの売上（単価 × 数量）を計算して、各商品のパフォーマンスを見極めることができます。
12  - 売上の高い商品や利益率の高い商品を特定することで、在庫管理やマーケティング戦略に活用できます。
13                                                   
14                                        ### 4. 利益分析
15  - 売上から原価を引くことで、各商品の利益を計算できます。これにより、利益率の高い商品や、原...
16                                                   
17                          

In [6]:
# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

In [7]:
# ワークフロー化
print("処理を開始します。")

# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')

# 2. データをLLM用にテキスト形式に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"

# 3. OpenAI APIの呼び出し
# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"
# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])

# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

print("Excelファイルに分析結果を保存しました。")

処理を開始します。
Excelファイルに分析結果を保存しました。
